In [3]:
import os
import json
import pickle
import requests
import numpy as np
import pandas as pd
from tmdbv3api import TMDb

from tmdbv3api import Movie

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [4]:
data = pd.read_csv("./final_data.csv")
data.head()

,movie_title,genres,director_name,actor_1_name,actor_2_name,actor_3_name,movie_feature
0,avatar,Action Adventure Fantasy Sci-Fi,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,CCH Pounder Joel David Moore Wes Studi James C...
1,pirates of the caribbean: at world's end,Action Adventure Fantasy,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,spectre,Action Adventure Thriller,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,the dark knight rises,Action Thriller,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,star wars: episode vii - the force awakens,Documentary,Doug Walker,Doug Walker,Rob Walker,unknown,Doug Walker Rob Walker unknown Doug Walker Doc...


In [5]:
count_vectorizer = CountVectorizer()
count_matrix = count_vectorizer.fit_transform(data["movie_feature"])
similarity_matrix = cosine_similarity(count_matrix)

In [6]:
def convert(o):
    if isinstance(o, np.generic):
        return o.item()
    raise TypeError

In [7]:
tmdb = TMDb()
TMDB_API_KEY = "eac61d7ffa1608d3e0188068092bd341"
tmdb.api_key = TMDB_API_KEY

In [8]:
def movie_search_engine(movie_name, TMDB_API_KEY = TMDB_API_KEY):
    MOVIE_SEARCH_LINK = "https://api.themoviedb.org/3/movie/"
    BASE_IMAGE_URL = "https://image.tmdb.org/t/p/original"
    tmdb_movie = Movie()
    search_result = tmdb_movie.search(movie_name.strip())
    movie_id = search_result[0]["id"]
    response = requests.get(MOVIE_SEARCH_LINK+"{}?api_key={}".format(movie_id, tmdb.api_key))
    json_response = response.json()
    # print(json_response)
    movie_data = {}
    movie_data["title"] = json_response["original_title"] if json_response["original_title"] and json_response["original_title"].lower()==movie_name else json_response["title"]
    movie_data["poster"] = BASE_IMAGE_URL + json_response["poster_path"]
    movie_data["overview"] = json_response["overview"]
    genres = []
    for genre in json_response["genres"]:
        genres.append(genre["name"])
    movie_data["genres"] = "*".join(genres)
    movie_data["rating"] = json_response["vote_average"]
    movie_data["total_votes"] = json_response["vote_count"]
    movie_data["tag_line"] = json_response["tagline"]
    movie_data["status"] = json_response["status"]
    return movie_data

In [9]:
def movie_recommender_engine(movie_name, n_top_recommendations=10):
    movie_name = movie_name.strip().lower()
    if movie_name not in data["movie_title"].unique():
        return json.dumps({"error": "Sorry! Movie is not in our database. Please check the spelling or try with another movie name"}, default=convert)
    else:
        index = data.loc[data["movie_title"] == movie_name].index[0]
        matrix = list(enumerate(similarity_matrix[index]))
        matrix = sorted(matrix, key=lambda x:x[1], reverse=True)
        recommended_indexes = [index for (index, similarity) in matrix[1:n_top_recommendations+1]]
        recommended_movies = {}
        for i, index in enumerate(recommended_indexes):
            r_movie_name = data["movie_title"][index]
            movie_data = movie_search_engine(r_movie_name)
            recommended_movies["movie"+str(i+1)] = movie_data
        recommended_movies = json.dumps(recommended_movies, default = convert)
        return recommended_movies

In [14]:
movie_recommender_engine("Triangle", 10)

'{"movie1": {"title": "The Messengers", "poster": "https://image.tmdb.org/t/p/original/kLmurNcnkmdPahLc6HKQjX3EHed.jpg", "overview": "When the Solomons trade in the craziness of big-city life for the quiet of a North Dakota farm, little do they expect the nightmare that follows. Soon after arriving, teenage Jess (Kristen Stewart) and her younger brother see terrifying apparitions and endure attacks from a supernatural source. Jess must warn her disbelieving family before it is too late to save them.", "genres": "Mystery*Thriller*Horror", "rating": 5.7, "total_votes": 616, "tag_line": "There is evidence to suggest that children are highly susceptible to paranormal phenomena. They see what adults cannot. They believe what adults deny. And they are trying to warn us.", "status": "Released"}, "movie2": {"title": "Saw", "poster": "https://image.tmdb.org/t/p/original/pparEGTCxcTiImyVFoKD0LX11N8.jpg", "overview": "Obsessed with teaching his victims the value of life, a deranged, sadistic seri